### Custom variable pattern

In [6]:
import gsconfig

# Кастомный паттерн распознавания переменных в шаблоне
pattern = r'\{%\s*([a-z0-9_!]+)\s*%\}'
template_body = """
- {% substring %} внутри строки
- Строка подстрока в кавычках: "{% variable %}"
- Явное указание на строку: {% substring!string %}
- Сколько коров?
- Вот столько: {% cow!int %} (int) или {% cow %} (source) 
- И еще давай словарик
- Вот словарик: {% dict %}
- И пустота: "{% empty %}"
"""
template = gsconfig.Template(body=template_body, pattern=pattern)

# Словарь с переменными для замены
data = {
    'variable': 'Woohooo!',
    'substring': 'subSTRing',
    'cow': 10.9,
    'dict': {'key': 'value', 'list': [{'one': 'odin'}, 'two', ['three', 'four', 'five']]},
    'empty': ''
}

r = template.make(data)
print(r)



- subSTRing внутри строки
- Строка подстрока в кавычках: "Woohooo!"
- Явное указание на строку: "subSTRing"
- Сколько коров?
- Вот столько: 10 (int) или 10.9 (source) 
- И еще давай словарик
- Вот словарик: {"key": "value", "list": [{"one": "odin"}, "two", ["three", "four", "five"]]}
- И пустота: ""



### Default variable pattern + template file

In [12]:
import gsconfig
import json

template_path = 'template.template'
template = gsconfig.Template(path=template_path)

# Словарь с переменными для замены
data = {
    'variable': 'Woohooo!',
    'substring': 'subSTRing',
    'cow': 10.9,
    'dict': {'key': 'value', 'list': [{'one': 'odin'}, 'two', ['three', 'four', 'five']]},
    'empty': ''
}

# r = json.loads(template.make(data))
r = template.make(data)
print(r)



{
    "string": "Woohooo!",
    "string_with_command": "Woohooo!",
    "string_as_substring": "Звучит как subSTRing. Похоже?",
    "int_as_substring": "Звучит как 10.9?",
    "digit_int": 10,
    "digit_source": 10.9,
    "dict": {"key": "value", "list": [{"one": "odin"}, "two", ["three", "four", "five"]]},
    "empty": ""
}
